https://grouplens.org/datasets/movielens/

In [ ]:
!pip install torch_tb_profiler
!pip install pytorch-lightning
!pip install mlflow
!pip install lightfm
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 623.0/623.0 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#@title Common Imports
import os
import subprocess
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.profiler as profiler
import torch.optim as optim
from torch import utils
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

import pytorch_lightning as pl

import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

# nice data table exploration in collab https://colab.research.google.com/notebooks/data_table.ipynb#scrollTo=jcQEX_3vHOUz
from google.colab import data_table

data_table.enable_dataframe_formatter()

In [ ]:
# THIS IS THE WORST MOST ROUND-ABOUT CODE IVE EVER WRITTEN
# I DIDNT NEED A DL I KNEW THAT AND FOR SOME REASON I DID IT ANYWAYS
import numpy as np
from lightfm.datasets import fetch_movielens

class FMMovielens(Dataset):
  def __init__(self, split):
    movielens = fetch_movielens()
    self.sparse = movielens[split]

  def __len__(self):
    return self.sparse.row.size

  def __getitem__(self, idx):
    return self.sparse.row[idx], self.sparse.col[idx], self.sparse.data[idx]

batch_size = 64
train_dl = DataLoader(FMMovielens('train'), batch_size=batch_size)
test_dl = DataLoader(FMMovielens('test'), batch_size=batch_size, shuffle=True)

In [ ]:
#train
movielens = fetch_movielens()
train = movielens['train']

model = LightFM(learning_rate=0.05, loss='bpr')
model.fit(train, epochs=30)

In [ ]:
#@title Testing for just basic loss

# model.eval()
with torch.no_grad():
  loss_func = nn.MSELoss()
  for batch in test_dl:
    user_ids, item_ids, ratings = batch
    pred_ratings = torch.tensor(model.predict(user_ids.numpy(), item_ids.numpy()))
    print(pred_ratings)
    loss = loss_func(pred_ratings, ratings.float())
    print(loss)-

tensor([-4.3011, -2.4934, -0.4726, -4.8038, -3.3726,  1.7917, -4.1602,  1.7176,
        -0.6797, -1.5867, -0.9009, -2.6440, -6.2419, -0.5707,  0.6802, -3.8830,
         0.0707, -0.5667,  0.8421, -6.5968, -5.8677, -1.9774, -2.4363,  0.5312,
         1.1251, -0.1799, -2.2303,  0.4909,  0.8236, -5.9382, -4.9484, -5.7894,
         0.8863, -1.7731, -3.5545, -0.6793, -1.4949, -1.0375, -0.5445, -1.7738,
        -0.0725, -4.5875, -6.0853, -2.2188, -2.7202, -4.5303, -5.5891, -3.4888,
        -4.6019, -5.6916, -6.0571, -3.7453,  0.0694, -3.6032, -0.3578, -5.3935,
        -6.0106, -0.0971, -3.3315,  0.4368,  0.4154, -4.0409,  0.5261,  2.0792])
tensor(41.8376)
tensor([ 1.0590,  0.7997, -1.5485,  2.1523, -2.0878, -2.3769, -0.5063, -4.4759,
        -3.3759,  0.9739,  0.5653, -4.8182, -3.3657, -2.4386, -2.2936, -1.1347,
         0.2974, -0.5953,  0.0631, -6.1146, -1.8902, -2.0806,  1.6869,  2.1651,
        -0.5713,  1.1281, -1.6900,  0.9368,  1.4908, -2.6583, -0.0800, -7.2486,
         0.8452, -2.458